# Link prediction - workshop maximum likelihood based methods

In this workshop we look at two methods for link prediction: **similarity based methods** and **maximum likelihood (ML) based methods**. The main idea behind similarity based methods, is that you choose a similarity metric (like the number of common neighbours) and base the reliability of an edge on the score from this metric. The art of this method is choosing a suitable similarity metric for your problem.

The idea behind ML methods is taking an underlying network model from which we assume the observed network was generated. Then, given the chosen model, we can compute the conditional probability that a certain edge is present in the graph given the observed network (using Bayes' rule). The higher this probability, the more likely it is that the link is missing. The art behind this method is choosing a suitable underlying random graph model your network is a realisation from. Sadly, ML methods are often difficult to implement. Therefore, we will look at one specific ML method in this workshop: one based on the **stochastic block model (SBM)**. 

**Exercise 1.** Give two reasons why the stochastic block model might be a good underlying model for real-life networks. Also, give two reasons why is might be poorly suited for real-life networks. 

## The likelihood function of the stochastic block model

The centrepiece of any ML method is the **likelihood function**. This function takes a network as input, and outputs the probability of this network appearing. For the stochastic block model, we know that each edge is present independently of the others with a probability that depends on the types of its adjacent nodes. If we know the stochastic block model's inputs $\vec{n}$ and $\textbf{P}$, then we can deduce that the likelihood function of a graph $G$ in this model is given by $$\mathbb{P}(G \;|\; \vec{n}, \textbf{P}) = \prod_{t \leq s} p_{ts}^{\ell_{ts}} (1 - p_{ts})^{r_{ts} - \ell_{ts}}.$$ Here, $\ell_{ts}$ is the number of links between vertex-types $t$ and $s$, and $r_{ts}$ is the maximum number of possible links between types $t$ and $s$. Finally, $p_{ts}$ is the entry at row $t$ and column $s$ of the matrix $\textbf{P}$, i.e. the probability that an edge between vertex-types $t$ and $s$ is present.

**Exercise 2.** Explain that the above likelihood function is correct. Then, implement it below. Test it on the generated stochastic block model underneath the function. *Hint: If implemented correctly, the likelihood should be approximately $6.25 \cdot 10^{-15}$.*

In [1]:
import networkx as nx
import scipy as sp
import numpy as np

def cLSBM(G, n, P):
    likelihood = 0
    
    # Your answer goes here
    
    return likelihood


#The code below tests your method

seed = 1 #Setting a random seed

# Generating the SBM
n = [5, 5, 5]
P = [[0.5, 0.1, 0.02], [0.1, 0.7, 0.1], [0.02, 0.1, 0.3]]
G = nx.stochastic_block_model(n, P, seed = 1)

# Computing likelihood
cLSBM(G, n ,P)

0

From Exercise 2 you should notice that the likelihood for a relatively small network is already quite low. This is to be expected, since you are multiplying many numbers that are smaller than $1$. However, since the numbers are so small, this will quickly cause problems when networks become big: the likelihood function will always output the value $0$. Hence, we often need tricks to ensure the likelihood remains meaningful. The idea behind all such tricks is noting that we do not need to use the pure conditional probability $\mathbb{P}(G \;|\; \vec{n}, \textbf{P})$ as our reliability. 

**Exercise 3$\star$.** Find a way to adapt your answer to Exercise 2 that is also useful for larger graphs. Implement it in the code block below, and test it on a larger SBM.

In [2]:
import networkx as nx
import scipy as sp
import numpy as np

def cLogLSBM(G, n, P):
    likelihood = 0
    
    # Your answer goes here
    
    return likelihood

# The code below tests your method

seed = 1 #Setting a random seed

# Generating the SBM
n = [50, 50, 50]
P = [[0.5, 0.1, 0.02], [0.1, 0.7, 0.1], [0.02, 0.1, 0.3]]
G = nx.stochastic_block_model(n, P, seed = seed)

# Computing likelihood
cLogLSBM(G, n ,P)

0

The likelihood function for the SBM we have now is relatively simple to use. Moreover, if $T_i$ is the type of vertex $i$ and $T_j$ the type of vertex $j$, then we also know that the likelihood of edge $\{i, j\}$ is given by entry $(T_i, T_j)$ in matrix $\textbf{P}$ (i.e., the value $p_{T_i, T_j}$). However, this analysis suffers from a major problem: it assumes we know the underlying SBM *exactly*. When we observe a network, however, we do not know the underlying SBM. All matrices $\textbf{P}$ and vectors $\vec{n}$ are fair game. Hence, to find the true likelihood, we need to integrate over all possible $\textbf{P}$ and $\vec{n}$.

Let us first see what happens if we do not know $\textbf{P}$, but do know $\vec{n}$ (and the exact allocation of types to vertices). In that case, if we observe a graph $G^O$ the reliability/likelihood will be given by $\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, G^O).$ Since we know the value of $\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, \text{P}, G^O) = p_{T_i T_j}$ we want to use the *law of total probability* to condition on the values in $\text{P}$. This yields the following integral: $$\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, G^O) = \iint_{[0, 1]^g} \mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, \textbf{P}, G^O) \cdot \mathbb{P}(\textbf{P} \;|\; \vec{n}, G^O) \; \text{d}\textbf{P}.$$
Here, $g$ is the number of distinct vertex-type pairs that are possible. Substituting what we know yields $$\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, G^O) = \iint_{[0, 1]^g} p_{T_i T_j} \cdot \mathbb{P}(\textbf{P} \;|\; \vec{n}, G^O) \; \text{d}\textbf{P}.$$

Now, to compute $\mathbb{P}(\textbf{P} \;|\; \vec{n}, G^O)$ we will use *Bayes' rule* to rewrite $$\mathbb{P}(\textbf{P} \;|\; \vec{n}, G^O) = \frac{\mathbb{P}(G^O \;|\; \vec{n}, \textbf{P} ) \mathbb{P}(\textbf{P}' \;|\; \vec{n})}{\iint_{[0, 1]^g} \mathbb{P}(G^O \;|\; \vec{n}, \textbf{P}' ) \mathbb{P}(\textbf{P}' \;|\; \vec{n}) \; \text{d}\textbf{P}'} = \frac{\prod_{t \leq s} p_{ts}^{\ell_{ts}} (1 - p_{ts})^{r_{ts} - \ell_{ts}} \mathbb{P}(\textbf{P} \;|\; \vec{n})}{\iint_{[0, 1]^g} \prod_{t \leq s} (p_{ts}')^{\ell_{ts}} (1 - (p_{ts}'))^{r_{ts} - \ell_{ts}} \mathbb{P}(\textbf{P}' \;|\; \vec{n}) \; \text{d}\textbf{P}'}.$$

Finally, we make the assumption that the values in the probability matrix $\textbf{P}'$ does not depend on the partitioning into groups $\vec{n}$. This means that $ \mathbb{P}(\textbf{P}' \;|\; \vec{n}) = c$ for some constant $c>0$ for all matrices $\textbf{P}'$. Hence, we find $$\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, G^O) = \iint_{[0, 1]^g} p_{T_i T_j} \cdot \frac{\prod_{t \leq s} p_{ts}^{\ell_{ts}} (1 - p_{ts})^{r_{ts} - \ell_{ts}} }{\iint_{[0, 1]^g} \prod_{t \leq s} (p_{ts}')^{\ell_{ts}} (1 - (p_{ts}'))^{r_{ts} - \ell_{ts}} \; \text{d}\textbf{P}'} \; \text{d}\textbf{P}.$$

Setting $C$ to be a (fixed) normalization constant, computing these integrals finally yields $$\mathbb{P}(\{i, j\} \in E \;|\; \vec{n}, G^O) = \frac{1}{C} \cdot \frac{\ell_{T_i T_j} + 1}{r_{T_i T_j} + 2} \prod_{t \leq s} \frac{1}{(r_{ts} + 1) \binom{r_{ts}}{\ell_{ts}}}.$$ 

**Exercise 4.** Look at the final expression after the derivaiton. Write down what is contained in the constant $C$, and explain why it is not needed to specify this number. Also, critically evaluate whether everything we do not "need" in the above expression is dumped into the constant $C$.

**Exercise 5.** Load the network ``link_prediction_ML1.gz``. It is given that this network was generated through a stochastic block model where the first $50$ vertices are of type $1$, the next $30$ of type $2$ and the final $20$ of type $3$. Write code to perform link prediciton on this network using the SBM ML method given the above likelihood function. Given that we removed link $\{14, 48\}$, are the results as expected?

In [3]:
import networkx as nx
import scipy as sp
import numpy as np

# Your answer goes here

Although the above derivation was already quite involved, you have probably noticed from Exercise 5 that giving away $\vec{n}$ and the exact location of each vertex-type is too much. There is not enough left to make the SBM ML link prediction method useful. Luckily (or to be fair sadly) we often also do not know (1) how many vertex-types there are, and (2) where these types are located. Thus, in reality the likelyhood will be given by $\mathbb{P}(\{i, j\} \in E \;|\; G^O)$. We can repeat the above caclulations to show that this will yields the following likelihood function for the stochastic block model: $$\mathbb{P}(\{i, j\} \in E \;|\; G^O) =  \frac{1}{C} \sum_{\textbf{T} \in \mathcal{T}} \frac{\ell_{T_i T_j} + 1}{r_{T_i T_j} + 2} \prod_{t \leq s} \frac{1}{(r_{ts} + 1) \binom{r_{ts}}{\ell_{ts}}}.$$ Here, the newly added sum forces us to compute the previous likelihood for all possible assignments of types to the vertices. So, $\mathcal{T}$ can be seen as the set of all type-assignments to the vertices in the graph, and $\textbf{T}$ can be seen as a signle sequence of vertex-types sampled from $\mathcal{T}$; we also call $\textbf{T}$ a **partition**. This is the likelihood function we need to make the SBM ML based method work. The question now becomes: how do we implement it?

**Exercise 6.** Look at the likelihood expression above, and answer the following questions. Write down your reasoning behind each answer.
- What is the value of $C$ in the likelihood expression above?
- Can the product in the likelihood expression above be taken out of the sum?
- Does the above likelihood function use global information about the graph $G^O$ or only local information around vertices $i$ and $j$?
- How many terms does the sum have? 
- How fast does the number of terms in the sum grow as $G^O$ has more vertices?

## Approximating the likelihood function of the SBM

We now know what the likelihood function of the SBM maximum likelihood method for link prediction looks like, so now we only have to compute this likelihood and we have found the reliability of each vertex. However, as we have seen in Exercise 6, computing the likelihood function is cumbersome and computationally unfeasible if the number of vertices in $G^O$ grows. Thus, we need to find a way to approximate the likelihood function. Since there are too many options to choose from, a first idea we can try, is fixing the number of vertex-types present in the graph and choosing partitions of vertices into groups uniformly at random. You then compute the sum of likelihoods over only those vertex types assignments you have sampled. 

In the rest of this workshop we will consider the (real-life) dataset in ``link_prediction_ML2.gz``. This dataset depicts books about US politics published around the 2004 election. An edge between two books means that the books have often been purchased together on "amazon.com". We have removed edge $\{40, 44\}$. 

**Exercise 7$\star$.** Think about the nature of the network ``link_prediction_ML2.gz``. How many vertex-types do you think there are? Use this knowledge to approximate the likelihood function, and do link prediciton using the SBM ML method on the dataset. How well does the method perform?

*Hint: You can use the skeleton below to guide you*

In [4]:
import networkx as nx
import numpy as np
import math

def twoClassSBMLinkPrediction(Adj, trials = 100, noTypes = 2):
    """
    Performs SBM link prediction by randomly sampling partitions with a given number of types.

    Parameters
    ----------
    Adj : Symmetric np.array() with {0, 1} entries;
        Adjacency matrix of the input graph.
    trials : int, optional;
        The number of partitions we sample. The default is 100.
    noTypes : int, optional;
        The number of vertex-types we assume. The default is 2.

    Returns
    -------
    reliability : np.array() with float entries;
        A matrix with the reliabilities of all edges.  
    """
    pReliability = np.zeros((trials, Adj.shape[0], Adj.shape[1]))
    
    for trial in range(trials):
        # Sample a random partition and compute number of vertices in each type
        
        
        # Compute edge dependent part of the likelihood function for each edge
        
        
        # Compute partition dependent, but edge independent part of the likelihood function
        
        continue
    
    # Sum over all computed reliabilities per partition
    reliability = np.sum(pReliability, axis = 0)
    
    return reliability

On of the problems you might have encountered in Exercise 7, is that the term $$f(\textbf{T}) = \prod_{t \leq s} \frac{1}{(r_{ts} + 1) \binom{r_{ts}}{\ell_{ts}}}$$ in the likelihood function is quite dominating. Hence, it is not very useful to saple partitions uniformly at random, since many of them will not contribute much to the final result anyways. We need to find a way to sample *relevant* partitions only. To do this, we apply the following idea:
1. Start with a random partition $\textbf{T}_{\text{old}}$.
2. Change the type of one of the vertices in $\textbf{T}_{\text{old}}$ randomly to create $\textbf{T}_{\text{new}}$.
3. Compute $f(\textbf{T}_{\text{new}})$. We accept the change in Step 2 (and set $\textbf{T}_{\text{old}}$ to be equal to $\textbf{T}_{\text{new}}$) if:
    * The value $f(\textbf{T}_{\text{new}})$ is larger than $f(\textbf{T}_{\text{old}})$, or
    * If $f(\textbf{T}_{\text{new}})$ is smaller than $f(\textbf{T}_{\text{old}})$, then accept the change only with probability $f(\textbf{T}_{\text{new}}) / f(\textbf{T}_{\text{old}})$.
4. After a large enough number of accepted changes, the value of $f(\textbf{T}_{\text{new}})$ should have converged to the value "important" partitions take. Now, we can start to sample partitions, and compute the likelihoods of them.
5. Repteat Steps 2 and 3 above until a fixed number of partition changes have been accepted.
6. Compute the likelihood of the partition you have obtained after Step 5 for all edges and store it.
7. Reteat Steps 5 and 6 until you have enough samples of the likelihood function. Sum over all obtained partition to find the reliability of each edge.

Another problem you might have faced in Exercise 7, is that the value $C$ was hard to guess and that the likelihood function was numerically unstable. There are some practicalities to keep in mind with regards to these issues:
* Working with $f(\textbf{T}_{\text{new}})$ directly is numerically unstable if you have many possible vertex-types. You might want to consider using only $\log(f(\textbf{T}_{\text{new}}))$.
* You need to choose the normalization constant $C$ in the likelihood function cleverly. It might be smart to save the "converged" value of $f(\textbf{T}_{\text{old}})$ and use that is your value of $C$, since you expect all subsequent evaluations of $f$ to be close to this value.

**Exercise 8.** Implement the above idea to do link prediction with the likelihood based stochastic block model method on ``link_prediction_ML2.gz``. How does it perform? To help you get started, below you will find a function that executes Step 1 through 4. Check this function thoroughly, so you understand what it does; there might still be tiny mistakes in it.

*Hint: You may fix the number of vertex-types, but you may also keep it unknown. The idea of the algorithm works in both settings, but the implementation is slightly simpler if you know the number of vertex-types.*

In [7]:
import networkx as nx
import numpy as np
import math



def initialConfigF(Adj, noTypes = 100, trainSteps = 1000):
    """
    Performs initial convergence of f(P) in the Metropolis algorithm.

    Parameters
    ----------
    Adj : Symmetric np.array() with {0, 1} entries;
        Adjacency matrix of the input graph.
    noTypes : int, optional;
        The number of communities we assume. The default is 100.
    trainSteps : int, optional;
        The number of iterative steps taken to converge f(P). The default is 1000.

    Returns
    -------
    Fval : float;
        The final (converged) value of log(f(P)). The logarithm is given to
        keep the algorithm numerically stable.
    oldP : np.array() with int entries;
        The partition P that produced f(P).

    """
    # Initial training for f
    Fval = - 10**10
    oldP = np.random.choice(noTypes,Adj.shape[0]) # Choose a random partition
    
    for step in range(trainSteps):
        print(step)
        fail = True
        
        # Keep trying new partition until you've found one that is accepted
        while fail:
            # Create new P
            index = np.random.randint(Adj.shape[0])
            newP = oldP.copy()
            newP[index] = np.random.choice(np.delete(np.arange(noTypes), oldP[index]))
    
            #Compute the logarithm of f(P) [for numerical stability]
            n = np.histogram(newP, bins = np.arange(noTypes + 1))[0]
            normconst = 0
            for k, val1 in enumerate(n):
                for l, val2 in enumerate(n[k:]):
                    l = k + l # Shift the index l to the correct value
                    rDummy = val1 * val2
                    c = Adj[newP == k, :]
                    d = c[:, newP == l]
                    ellDummy = np.sum(d)
                    if k == l:
                        rDummy = (val1 - 1)* val1 / 2
                        ellDummy = ellDummy / 2
                    if rDummy <= 50 or ellDummy == 0:
                        normconst += -np.log(rDummy + 1) - np.log(math.comb(int(rDummy), int(ellDummy)))
                    else:#Approximation of logarithm of binomial coefficient
                        normconst += -np.log(rDummy + 1) - (rDummy * np.log(rDummy) - ellDummy * np.log(ellDummy) - (rDummy - ellDummy) * np.log(rDummy - ellDummy))
            
            # Test whether we accept the change, and if accepted: update
            if np.random.binomial(1, min([1., np.exp(normconst - Fval)])) == 1:
                Fval = normconst
                oldP = newP.copy()
                fail = False
    return Fval, oldP

## Validation

In certain exercises you were asked to assess the performance of a link prediction algorithm. You have probably found out that this is not an easy task. In general, validation for link prediction is a difficult talk, since we do not know that the actual network looks like. Usually this lack of a ground truth is mediated by using **cross-validation** methods. The idea is to remove extra edges from a network, and using the link prediction method to assess how well these extra edges are retrieved. By doing this multiple times, and averaging over the results you can obtain a score that tells you how well the link prediction method performs on a given problem.

**Exercise 9.** The code below implements a specific cross-validation method for the Jaccard index (this is a similarity based link prediction method). Find out what the code does, and add comments to explain the procedure. Then, adapt it to the SBM maximum likelihood link prediction method, and use it to test the performance of this algorithm.

In [44]:
def leaveOneOutJaccard(G):
    AUC = 0
    for e in G.edges:
        Gprobed = G.copy()
        Gprobed.remove_edge(e[0],e[1])
        target = nx.jaccard_coefficient(Gprobed, [e])
        for _, _, p in target:
            reliabilityTarget = p
        
        reliability = nx.jaccard_coefficient(Gprobed)
        better = 0
        same = 0
        total = 0
        for _, _, p in reliability:
            if p < reliabilityTarget:
                better += 1
            elif p == reliabilityTarget:
                same += 1
            total += 1
        
        AUC += (0.5 * same + better) / total
    return AUC / len(G.edges)

In this workshop you have now tried the stochastic block model maximum likelihood method for link prediction on both real and synthetic data. You should now have noticed the following:
- Link prediction is a difficult task, and in practice it is almost impossible to retrieve the missing links exactly.
- Maximum likelihood methods are often difficult to implement and by the nature of the likelihood function numerically unstable.
- The performance of maximum likelihood methods heavily depends on how suitible your underlying null model is for the problem.
- You need to understand the stochastic structure of the underlying null model well if you want to use it for link predictoin.
- It is not easy to validate and assess performance of link prediction algorithms (more on this in a later lecture).